In [36]:
import random
from datasets import load_dataset


In [2]:
def shuffle_options_and_update_row(row):
    options = row['options']
    answer_index = row['answer_index']

    # Store the correct answer before shuffling
    correct_option = options[answer_index]

    # Shuffle the options
    shuffled_options = options[:]
    random.shuffle(shuffled_options)

    # Update the answer and answer_index
    new_answer_index = shuffled_options.index(correct_option)
    new_answer = chr(65 + new_answer_index)  # Convert index to letter (A, B, C, ...)

    # Update the row
    row['options'] = shuffled_options
    row['answer_index'] = new_answer_index
    row['answer'] = new_answer

    return row

def shuffle_options_and_update(test_df):
    res_df = []
    for each in test_df:
        res_df.append(shuffle_options_and_update_row(each))
    return res_df

In [37]:
def preprocess(test_df):
    res_df = []
    for each in test_df:
        options = []
        for opt in each["options"]:
            if opt == "N/A":
                continue
            options.append(opt)
        each["options"] = options
        #each = shuffle_options_and_update_row(each)
        # if random.random() < 0.5:
        #     each = modify_question(each)
        res_df.append(each)
    res = {}
    for each in res_df:
        if each["category"] not in res:
            res[each["category"]] = []
        res[each["category"]].append(each)
    return res

In [38]:
def load_mmlu_pro():
    dataset = load_dataset("TIGER-Lab/MMLU-Pro")
    test_df, val_df = dataset["test"], dataset["validation"]
    test_df = preprocess(test_df)
    val_df = preprocess(val_df)
    return test_df, val_df

In [39]:
test_df3, val_df3 = load_mmlu_pro()

In [33]:
test_df2['business'][6]

{'question_id': 76,
 'question': ' Pine and Gilmore (1999) derive four distinct realms of experience, based on two dimensions. What are these dimensions?',
 'options': ['Customer participation and environmental acquisition.',
  'Environmental acquisition and environmental relationship.',
  'Customer retention and environmental relationship.',
  'Customer participation and environmental relationship.',
  'Customer acquisition and customer retention.',
  'Customer participation and customer relationship.',
  'Customer acquisition and environmental participation.',
  'Environmental participation and customer relationship.',
  'Customer retention and customer relationship.',
  'Customer acquisition and environmental relationship.'],
 'answer': 'D',
 'answer_index': 3,
 'cot_content': '',
 'category': 'business',
 'src': 'ori_mmlu-marketing'}

In [34]:
test_df3['business'][6]

{'question_id': 76,
 'question': ' Pine and Gilmore (1999) derive four distinct realms of experience, based on two dimensions. What are these dimensions?',
 'options': ['Customer participation and environmental acquisition.',
  'Environmental acquisition and environmental relationship.',
  'Customer retention and environmental relationship.',
  'Customer acquisition and customer retention.',
  'Customer participation and customer relationship.',
  'Customer acquisition and environmental participation.',
  'Environmental participation and customer relationship.',
  'Customer retention and customer relationship.',
  'Customer acquisition and environmental relationship.',
  'None of the above are correct'],
 'answer': 'I',
 'answer_index': 9,
 'cot_content': '',
 'category': 'business',
 'src': 'ori_mmlu-marketing'}

In [6]:
list(test_df2.keys())

['business',
 'law',
 'psychology',
 'biology',
 'chemistry',
 'history',
 'other',
 'health',
 'economics',
 'math',
 'physics',
 'computer science',
 'philosophy',
 'engineering']

In [ ]:
test_df_2 = shuffle_options_and_update(test_df)

In [18]:
test_df

[{'question_id': 70,
  'question': 'Typical advertising regulatory bodies suggest, for example that adverts must not: encourage _________, cause unnecessary ________ or _____, and must not cause _______ offence.',
  'options': ['Safe practices, Fear, Jealousy, Trivial',
   'Unsafe practices, Distress, Joy, Trivial',
   'Safe practices, Wants, Jealousy, Trivial',
   'Safe practices, Distress, Fear, Trivial',
   'Unsafe practices, Wants, Jealousy, Serious',
   'Safe practices, Distress, Jealousy, Serious',
   'Safe practices, Wants, Fear, Serious',
   'Unsafe practices, Wants, Fear, Trivial',
   'Unsafe practices, Distress, Fear, Serious'],
  'answer': 'I',
  'answer_index': 8,
  'cot_content': '',
  'category': 'business',
  'src': 'ori_mmlu-business_ethics'},
 {'question_id': 71,
  'question': 'Managers are entrusted to run the company in the best interest of ________. Specifically, they have a duty to act for the benefit of the company, as well as a duty of ________ and of _______.',


In [24]:
for q in test_df2['business']:
    if q['question_id'] == 70:
        print(q)
        break

{'question_id': 70, 'question': 'Typical advertising regulatory bodies suggest, for example that adverts must not: encourage _________, cause unnecessary ________ or _____, and must not cause _______ offence.', 'options': ['Unsafe practices, Distress, Joy, Trivial', 'Safe practices, Distress, Fear, Trivial', 'Safe practices, Distress, Jealousy, Serious', 'Safe practices, Wants, Fear, Serious', 'Safe practices, Wants, Jealousy, Trivial', 'Unsafe practices, Distress, Fear, Serious', 'Unsafe practices, Wants, Fear, Trivial', 'Safe practices, Fear, Jealousy, Trivial', 'Unsafe practices, Wants, Jealousy, Serious'], 'answer': 'F', 'answer_index': 5, 'cot_content': '', 'category': 'business', 'src': 'ori_mmlu-business_ethics'}


In [1]:
import json
from collections import defaultdict

# Load the JSON data from a file
file_path = 'eval_results/model_outputs_Meta-Llama-3-8B-Instruct_5shots.json'  # Replace with your file path
with open(file_path, 'r') as file:
    data = json.load(file)

# Initialize dictionaries to track correct answers and total questions per category
correct_answers = defaultdict(int)
total_questions = defaultdict(int)

# Calculate accuracy per category
for entry in data:
    category = entry['category']
    total_questions[category] += 1
    if entry['answer'] == entry['pred']:
        correct_answers[category] += 1

# Compute and display accuracy
accuracy_per_category = {}
for category in total_questions:
    accuracy_per_category[category] = correct_answers[category] / total_questions[category] * 100

# Print accuracy per category
for category, accuracy in accuracy_per_category.items():
    print(f"Category: {category}, Accuracy: {accuracy:.2f}%")


Category: business, Accuracy: 39.67%
Category: law, Accuracy: 26.34%
Category: psychology, Accuracy: 59.40%
Category: biology, Accuracy: 66.25%
Category: chemistry, Accuracy: 25.53%
Category: history, Accuracy: 42.26%
Category: other, Accuracy: 45.89%
Category: health, Accuracy: 48.90%
Category: economics, Accuracy: 53.20%
Category: math, Accuracy: 33.31%
Category: physics, Accuracy: 33.26%
Category: computer science, Accuracy: 42.20%
Category: philosophy, Accuracy: 40.08%
Category: engineering, Accuracy: 30.65%


In [28]:
def modify_question(question_data):
    # Remove the correct option based on the answer_index
    for i, option in enumerate(question_data["options"]):
        if "none" in option.lower() and "above" in option.lower():
            return question_data
        
    correct_index = question_data["answer_index"]  # Convert to 0-based index
    question_data["options"].pop(correct_index)
    
    # Add "None of the above are correct" as a new option
    question_data["options"].append("None of the above are correct")
    
    # Update the answer_index and answer
    question_data["answer_index"] = len(question_data["options"]) - 1  # Last option is now the answer
    question_data["answer"] = chr(64 + question_data["answer_index"])  # Convert index to letter (A, B, C...)
    
    return question_data

In [21]:
random.random()

0.6767893663825661